In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Psoriatic_Arthritis"
cohort = "GSE141934"

# Input paths
in_trait_dir = "../../input/GEO/Psoriatic_Arthritis"
in_cohort_dir = "../../input/GEO/Psoriatic_Arthritis/GSE141934"

# Output paths
out_data_file = "../../output/preprocess/Psoriatic_Arthritis/GSE141934.csv"
out_gene_data_file = "../../output/preprocess/Psoriatic_Arthritis/gene_data/GSE141934.csv"
out_clinical_data_file = "../../output/preprocess/Psoriatic_Arthritis/clinical_data/GSE141934.csv"
json_path = "../../output/preprocess/Psoriatic_Arthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the series summary and design, this dataset contains transcriptional data 
# which implies gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait - looking at diagnosis information in rows 5 and 6
# Row 6 (working_diagnosis) contains Psoriatic Arthritis data
trait_row = 6

# For age - found in row 2 
age_row = 2

# For gender - found in row 1
gender_row = 1

# 2.2 Data Type Conversion Functions

# Function to convert trait data to binary (1 for Psoriatic Arthritis, 0 for others)
def convert_trait(value):
    if not value or ':' not in value:
        return None
    diagnosis = value.split(':', 1)[1].strip()
    if diagnosis == "Psoriatic Arthritis":
        return 1
    return 0

# Function to convert age data to continuous values
def convert_age(value):
    if not value or ':' not in value:
        return None
    try:
        age = int(value.split(':', 1)[1].strip())
        return age
    except (ValueError, TypeError):
        return None

# Function to convert gender data to binary (0 for female, 1 for male)
def convert_gender(value):
    if not value or ':' not in value:
        return None
    gender = value.split(':', 1)[1].strip()
    if gender.upper() == 'F':
        return 0
    elif gender.upper() == 'M':
        return 1
    return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
# Initial filtering on dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction - Only if trait_row is not None
if trait_row is not None:
    # The sample characteristics dictionary represents characteristics categorized by row index
    # First, we need to create a proper clinical data DataFrame
    
    # Get the sample characteristics dictionary from the previous step
    sample_char_dict = {0: ['patient: 1072', 'patient: 1085', 'patient: 1076', 'patient: 1087', 'patient: 1080', 'patient: 1088', 'patient: 1083', 'patient: 1094', 'patient: 1050', 'patient: 1067', 'patient: 1051', 'patient: 1054', 'patient: 1070', 'patient: 1058', 'patient: 2010', 'patient: 2012', 'patient: 2029', 'patient: 2075', 'patient: 2062', 'patient: 2078', 'patient: 2086', 'patient: 2087', 'patient: 2067', 'patient: 2072', 'patient: 2090', 'patient: 1019', 'patient: 1020', 'patient: 1003', 'patient: 1008', 'patient: 2030'], 
                          1: ['gender: F', 'gender: M'], 
                          2: ['age: 50', 'age: 43', 'age: 66', 'age: 55', 'age: 52', 'age: 54', 'age: 63', 'age: 61', 'age: 58', 'age: 79', 'age: 69', 'age: 57', 'age: 46', 'age: 44', 'age: 59', 'age: 81', 'age: 60', 'age: 92', 'age: 45', 'age: 47', 'age: 27', 'age: 38', 'age: 51', 'age: 70', 'age: 56', 'age: 53', 'age: 74', 'age: 49', 'age: 31', 'age: 65'], 
                          3: ['tissue: peripheral blood'], 
                          4: ['cell type: CD4+ T cells'], 
                          5: ['first_diagnosis: Rheumatoid Arthritis', 'first_diagnosis: Undifferentiated Inflammatory Arthritis', 'first_diagnosis: Reactive Arthritis', 'first_diagnosis: Crystal Arthritis', 'first_diagnosis: Psoriatic Arthritis', 'first_diagnosis: Non-Inflammatory', 'first_diagnosis: Other Inflammatory Arthritis', 'first_diagnosis: Enteropathic Arthritis', 'first_diagnosis: Undifferentiated Spondylo-Arthropathy', 'first_diagnosis: Unknown'], 
                          6: ['working_diagnosis: Rheumatoid Arthritis', 'working_diagnosis: Psoriatic Arthritis', 'working_diagnosis: Reactive Arthritis', 'working_diagnosis: Crystal Arthritis', 'working_diagnosis: Osteoarthritis', 'working_diagnosis: Non-Inflammatory', 'working_diagnosis: Undifferentiated Inflammatory Arthritis', 'working_diagnosis: Other Inflammatory Arthritis', 'working_diagnosis: Enteropathic Arthritis', 'working_diagnosis: Undifferentiated Spondylo-Arthropathy', 'working_diagnosis: Unknown']}
    
    # Create a DataFrame where each row is a patient and each column is a feature
    # We'll extract values from individual cells in the dictionary and create a more suitable structure
    
    # For the purposes of geo_select_clinical_features, we need a DataFrame where:
    # - Rows correspond to features (rows in the sample characteristics dictionary)
    # - Each column represents a sample
    clinical_data = pd.DataFrame(sample_char_dict)
    
    # Extract clinical features using the geo_select_clinical_features function
    selected_clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_features)
    print("Clinical Features Preview:")
    print(preview)
    
    # Save the extracted clinical features as a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to: {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I've analyzed the errors - it appears the input files don't exist in the expected directory structure. Let me implement a more robust solution that adapts to the available data.

```python
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any
import glob
import re
from pathlib import Path

# Initialize variables
sample_characteristics = {}
background = {}
clinical_data = pd.DataFrame()

# Check directory contents to find available files
print(f"Checking contents of {in_cohort_dir}")
if os.path.exists(in_cohort_dir):
    files = os.listdir(in_cohort_dir)
    print(f"Files in directory: {files}")
else:
    print(f"Directory {in_cohort_dir} does not exist")
    # Try to check if parent directory exists
    parent_dir = os.path.dirname(in_cohort_dir)
    if os.path.exists(parent_dir):
        print(f"Parent directory {parent_dir} exists with contents: {os.listdir(parent_dir)}")

# Try multiple possible paths for sample characteristics
possible_paths = [
    os.path.join(in_cohort_dir, "sample_characteristics.json"),
    os.path.join(in_trait_dir, "sample_characteristics.json"),
    os.path.join(in_cohort_dir, "characteristics.json"),
    os.path.join(in_cohort_dir, "samples.json")
]

for path in possible_paths:
    if os.path.exists(path):
        print(f"Found sample characteristics at {path}")
        with open(path, 'r') as f:
            sample_characteristics = json.load(f)
        break
else:
    print("Could not find sample characteristics file")

# Try multiple possible paths for background info
possible_bg_paths = [
    os.path.join(in_cohort_dir, "background.json"),
    os.path.join(in_trait_dir, "background.json"),
    os.path.join(in_cohort_dir, "metadata.json"),
    os.path.join(in_cohort_dir, "info.json")
]

for path in possible_bg_paths:
    if os.path.exists(path):
        print(f"Found background info at {path}")
        with open(path, 'r') as f:
            background = json.load(f)
        break
else:
    print("Could not find background information file")

# Look for any CSV file that might contain clinical data
csv_files = glob.glob(os.path.join(in_cohort_dir, "*.csv"))
if csv_files:
    # Try to identify the most likely clinical data file
    for file in csv_files:
        if "clinical" in file.lower() or "pheno" in file.lower() or "characteristic" in file.lower():
            print(f"Found clinical data at {file}")
            clinical_data = pd.read_csv(file)
            break
    else:
        # If no specific clinical file found, use the first CSV
        print(f"Using first CSV file as clinical data: {csv_files[0]}")
        clinical_data = pd.read_csv(csv_files[0])
else:
    # Try parent directory
    csv_files = glob.glob(os.path.join(in_trait_dir, "*.csv"))
    if csv_files:
        for file in csv_files:
            if os.path.basename(file).startswith(cohort) or cohort in file:
                print(f"Found possible clinical data at {file}")
                clinical_data = pd.read_csv(file)
                break

# Determine gene data availability based on available information
is_gene_available = True  # Default assumption

# Check platform info in background data if available
if background and "platform" in background:
    platform = str(background["platform"]).lower()
    if "mirna" in platform or "methylation" in platform:
        is_gene_available = False
    print(f"Platform info: {platform}")
else:
    # Check file names for clues about data type
    expression_files = [f for f in files if os.path.exists(in_cohort_dir) and 
                        ("expression" in f.lower() or "gene" in f.lower())]
    if not expression_files:
        # If no expression files and we have CSV files that might be miRNA or methylation
        for f in csv_files:
            if "mirna" in f.lower() or "methylation" in f.lower():
                is_gene_available = False
                break

# Initialize trait, age, and gender rows
trait_row = None
age_row = None
gender_row = None

# Examine the sample characteristics to identify relevant rows
if sample_characteristics:
    print("Sample Characteristics Keys:")
    for key, values in sample_characteristics.items():
        if not values:
            continue
        
        # Get a sample of unique values for display
        unique_values = list(set(str(v) for v in values if v is not None))[:5]
        print(f"Key {key}: {unique_values}")
        
        # Check for psoriatic arthritis related information
        if any(re.search(r'psoria|arthritis|psa', str(v).lower()) for v in values):
            trait_row = int(key)
        
        # Check for age information
        if any(re.search(r'age|years old', str(v).lower()) for v in values):
            age_row = int(key)
        
        # Check for gender/sex information
        if any(re.search(r'gender|sex', str(v).lower()) for v in values):
            gender_row = int(key)
else:
    print("No sample characteristics data available")

# Define conversion functions
def convert_trait(value):
    """Convert trait values to binary (0 for control, 1 for psoriatic arthritis)"""
    if value is None:
        return None
    
    value_lower = str(value).lower()
    if ":" in value_lower:
        value_lower = value_lower.split(":", 1)[1].strip()
    
    if re.search(r'psoria.*arthritis|psa', value_lower):
        return 1
    elif re.search(r'control|healthy|hc', value_lower):
        return 0
    return None

def convert_age(value):
    """Convert age values to continuous numeric values"""
    if value is None:
        return None
    
    value_str = str(value).lower()
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    
    # Extract numeric age
    age_match = re.search(r'(\d+\.?\d*)', value_str)
    if age_match:
        try:
            return float(age_match.group(1))
        except:
            return None
    return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    
    value_str = str(value).lower()
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    
    if re.search(r'female|f$|f\s', value_str):
        return 0
    elif re.search(r'male|m$|m\s', value_str):
        return 1
    return None

# Determine if trait data is available
is_trait_available = trait_row is not None

# Print findings
print(f"Is gene data available: {is_gene_available}")
print(f"Is trait data available: {is_trait_available}")
print(f"Trait row: {trait_row}")
print(f"Age row: {age_row}")
print(f"Gender row: {gender_row}")

# Save metadata about the cohort
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if is_trait_available and not clinical_


### Step 4: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll implement the code that properly accesses the available data for this cohort.

```python
import pandas as pd
import json
import os
from typing import Callable, Optional, Dict, Any
import glob

# First, let's explore the input directory to see what files are available
print(f"Contents of input directory {in_cohort_dir}:")
directory_contents = os.listdir(in_cohort_dir)
for item in directory_contents:
    print(f"  - {item}")

# Look for potential clinical or sample data files
clinical_file_candidates = glob.glob(os.path.join(in_cohort_dir, "*clinical*.csv"))
clinical_file_candidates.extend(glob.glob(os.path.join(in_cohort_dir, "*clinical*.pkl")))
clinical_file_candidates.extend(glob.glob(os.path.join(in_cohort_dir, "*sample*.csv")))
clinical_file_candidates.extend(glob.glob(os.path.join(in_cohort_dir, "*sample*.pkl")))

print("\nPotential clinical data files:")
for file in clinical_file_candidates:
    print(f"  - {file}")

# Try to load from clinical_data.csv (which might have been generated in a previous step)
try:
    clinical_data_paths = [
        os.path.join(in_cohort_dir, "clinical_data.csv"),
        os.path.join(in_cohort_dir, "sample_characteristics.csv"),
        os.path.join(in_cohort_dir, "clinical_data.pkl"),
        os.path.join(in_cohort_dir, "GSE141934_clinical_data.csv")
    ]
    
    clinical_data = None
    data_path_used = None
    
    for path in clinical_data_paths:
        if os.path.exists(path):
            if path.endswith('.csv'):
                clinical_data = pd.read_csv(path, index_col=0)
            else:
                clinical_data = pd.read_pickle(path)
            data_path_used = path
            break
    
    if clinical_data is not None:
        print(f"\nClinical data loaded from {data_path_used}")
        
        # Display the clinical data to understand its structure
        print("\nClinical data shape:", clinical_data.shape)
        print("\nClinical data preview:")
        print(clinical_data.head())
        
        # Get unique values for each row to identify relevant rows
        unique_values_dict = {}
        for idx, row in clinical_data.iterrows():
            unique_values = set(row)
            unique_values_dict[idx] = unique_values
            if len(unique_values) <= 20:  # Only show if reasonable number of unique values
                print(f"Row {idx}: {unique_values}")
        
        # 1. Gene Expression Data Availability - Assume it's available based on cohort
        is_gene_available = True
        
        # 2. Variable Availability and Data Type Conversion
        # Identify relevant rows for trait, age, and gender
        trait_row = None
        age_row = None
        gender_row = None
        
        for idx, unique_vals in unique_values_dict.items():
            values_str = ' '.join(str(val).lower() for val in unique_vals if val is not None)
            
            # Look for trait/diagnosis row
            if ('psoriatic' in values_str and 'arthritis' in values_str) or ('psa' in values_str and ('healthy' in values_str or 'control' in values_str)):
                trait_row = idx
                print(f"Found trait row at index {idx}")
            
            # Look for age row
            if 'age' in values_str:
                age_row = idx
                print(f"Found age row at index {idx}")
            
            # Look for gender row
            if 'female' in values_str or 'male' in values_str or 'gender' in values_str or 'sex' in values_str:
                gender_row = idx
                print(f"Found gender row at index {idx}")
        
        # 2.2 Data Type Conversion Functions
        def convert_trait(value: str) -> int:
            """Convert trait values to binary (0 for control, 1 for disease)"""
            if value is None or pd.isna(value) or value == '':
                return None
            
            value_str = str(value).lower()
            
            # Extract value after colon if present
            if ':' in value_str:
                value_str = value_str.split(':', 1)[1].strip()
            
            if 'healthy' in value_str or 'control' in value_str or 'hc' in value_str:
                return 0
            elif 'psoriatic' in value_str or 'psa' in value_str or 'patient' in value_str:
                return 1
            return None
        
        def convert_age(value: str) -> float:
            """Convert age values to float"""
            if value is None or pd.isna(value) or value == '':
                return None
            
            value_str = str(value).lower()
            
            # Extract value after colon if present
            if ':' in value_str:
                value_str = value_str.split(':', 1)[1].strip()
            
            # Extract numeric part
            import re
            age_match = re.search(r'(\d+)', value_str)
            if age_match:
                return float(age_match.group(1))
            return None
        
        def convert_gender(value: str) -> int:
            """Convert gender values to binary (0 for female, 1 for male)"""
            if value is None or pd.isna(value) or value == '':
                return None
            
            value_str = str(value).lower()
            
            # Extract value after colon if present
            if ':' in value_str:
                value_str = value_str.split(':', 1)[1].strip()
            
            if 'female' in value_str or 'f' in value_str:
                return 0
            elif 'male' in value_str or 'm' in value_str:
                return 1
            return None
        
        # 3. Save Metadata
        is_trait_available = trait_row is not None
        validate_and_save_cohort_info(
            is_final=False,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available
        )
        
        # 4. Clinical Feature Extraction
        if is_trait_available:
            # Extract clinical features
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=convert_gender
            )
            
            # Preview the extracted clinical features
            print("\nPreview of selected clinical features:")
            preview = preview_df(selected_clinical_df)
            print(preview)
            
            # Save clinical data to CSV
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            selected_clinical_df.to_csv(out_clinical_data_file)
            print(f"Clinical data saved to {out_clinical_data_file}")
        else:
            print("Trait data is not available. Skipping clinical feature extraction.")
    
    else:
        print("No clinical data files found in expected locations.")
        # If we can't find the clinical data, we still need to save metadata
        is_gene_available = True  # Assuming gene expression data is available
        is_trait_available = False  # No trait information available
        
        validate_and_save_cohort_info(
            is_final=False,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait


### Step 5: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 6: Gene Identifier Review

In [ ]:
# Based on the identifiers shown, these are Illumina BeadArray probe IDs (ILMN_*), 
# not human gene symbols. These identifiers need to be mapped to official gene symbols.
# The "ILMN_" prefix indicates these are Illumina's proprietary identifiers for
# their microarray probes.

requires_gene_mapping = True


### Step 7: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 8: Gene Identifier Mapping

In [ ]:
# 1. Identify which columns in gene_annotation contain probe IDs and gene symbols
# From the preview, we can see 'ID' contains the probe identifiers (ILMN_*)
# and 'Symbol' contains the gene symbols
prob_col = 'ID'
gene_col = 'Symbol'

# 2. Get gene mapping dataframe using the get_gene_mapping function
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping shape: {gene_mapping.shape}")
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level data to gene expression data
# Using the apply_gene_mapping function that handles many-to-many relations
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])

# 4. Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# 5. Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


### Step 9: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# Re-extract clinical features since we need it for linking
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# 2. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function from the library.
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape before handling missing values: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from endothelial cells derived from circulating progenitors of RA patients"
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    print(f"Data is usable. Saving to {out_data_file}")
    unbiased_linked_data.to_csv(out_data_file)
else:
    print("Data is not usable. Not saving linked data file.")